# Еще ООП

## Метаклассы

Вспоминаем, что в питоне все является объектом. В том числе классы. 

In [ ]:
class MyClass: # этот код создает в памяти объект на который ссылается переменная MyClass
    def __init__(self, a):
        print('Created instance of MyClass!')
        self.a = a

Объект `MyClass` может сам порождать объекты --> является классом.

In [ ]:
my_obj = MyClass(1)

Объект ```my_obj``` уже не может порождать объекты, потому что является экземпляром класса, но не классом!

При этом с классом можно делать все то же самое, что и с любым объектом:

In [ ]:
# записать в переменную
class_to_make = MyClass
my_obj = class_to_make(1)

In [ ]:
my_obj.a

In [ ]:
# передать в функцию 
def create_instance(class_object, a=1):
    print(a)
    return class_object(a)

In [ ]:
my_obj = create_instance(MyClass, 1)

In [ ]:
my_obj.a

In [ ]:
# добавить или изменить атрибут (это будет атрибут класса)
MyClass.new_class_attr = 10
my_obj1 = MyClass(1)

In [ ]:
my_obj1.new_class_attr

In [ ]:
MyClass.new_class_attr = 20

In [ ]:
my_obj1.new_class_attr

### Динамическое создание классов

Классы можно создавать динамически с помощью `type`, передав следующие аргументы: имя нового класса, кортеж с объектами родительских классов, словарь с названиями и значениями атрибутов классов. 

In [ ]:
class Animal:
   
    def __init__(self, name, legs, scariness):
        self.name = name 
        self.legs = legs
        self.scariness = scariness
        
class Cat(Animal):
    def sound(self): 
        print('Meow!')

In [ ]:
Kitten = type('Kitten', # имя нового класса
                (Cat, ), # кортеж с родительскими классами
                {'is_smol':True}) # словарь с названиями и значениями атрибутов классов

In [ ]:
help(Kitten)

In [ ]:
kitten = Kitten(name='Small Nice Kitty', legs=4, scariness=-10)

In [ ]:
kitten.name

In [ ]:
kitten.sound()

In [ ]:
kitten.is_smol

Добавление методов в класс:

In [ ]:
# создаем функцию (сallable object) с нужныи именем и аргументами
def introduce_kitten(self):
    return "Hello I am %s!" % (self.name)

In [ ]:
# и передаем в уже готовый класс
# метод необязательно называть так же, как и функцию, это неважно
# потому что мы просто присваиваем значение одной переменной другой переменной
Kitten.introduce = introduce_kitten

In [ ]:
kitten.introduce()

По сути методы  - это атрибуты являющиеся Callable (которые можно вызывать). И обращаемся мы с ними точно так же, как с просто атрибутами. 

In [ ]:
callable(introduce_kitten)

In [ ]:
# или передаем в словаре аргументов при создании класса
Kitten = type('Kitten', # имя нового класса
                (Cat, ), # кортеж с родительскими классами
                {'is_smol':True, 'introduce': introduce_kitten}) # словарь с названиями и значениями атрибутов классов

Можно создавать классы в цикле:

In [ ]:
def introduce(self):
    return "Hello I am %s, %s year student!" % (self.name, str(self.year))

new_classes = []
for i in range(1,5):
    new_classes.append(type('Student_%s_year'%str(i), (object, ), {'year': i, 'introduce': introduce})) 

In [ ]:
names = ['Vasya', 'Masha', 'Petya', 'Dasha']
for i, class_ in enumerate(new_classes):
    obj = class_()
    obj.name = names[i] # атрибуты экземпляра задаются отдельно
    print(obj)
    print(obj.year)
    print(obj.introduce()+'\n')

### Что такое меткласс?

То же самое можно сделать с помощью метакласса.    
**Метакласс** - класс, экземпляры которого сами являются классами (могут порождать свои экземпляры). 
![](https://blog.ionelmc.ro/2015/02/09/understanding-python-metaclasses/instance-of.png)

#### \_\_new\_\_ и \_\_init\_\_ 

![](https://i.stack.imgur.com/MgRbx.png)
+ `__new__()` - отвечает за создание нового экземпляра класса, возвращает новый объект (**должен быть return**)
+ `__init__()` - отвечает за инициализацию нового экземпляра класса - объявить какие у него есть атрибуты, какие у них значения (**без returna**)


In [ ]:
class MyClass:

    def __init__(self):
        self.my_attr = 1
        print('init called')

    def __new__(self, *args, **kwargs):
        print('new called')
        return super().__new__(self, *args, **kwargs)


In [ ]:
new_instance = MyClass()
print(new_instance.my_attr)

Создаем метакласс, который будет порождать классы "студент Х-го курса":

In [ ]:
# переопределим конструктор, чтобы атрибут name передавался в качестве аргумента
def __init__(self, name):
    self.name = name

In [ ]:
# создадим метод introduce
def introduce(self):
    return "Hello I am %s, %s year student!" % (self.name, str(self.year))

In [ ]:
# допустим мы хотим отдельно передавать методы в виде списка
# и так, чтобы они автоматически добавлялись с нужным именем
student_methods = [introduce, __init__]

In [ ]:
class StudentMetaClass(type): # обязательно наследуемся от type 
    def __new__(cls, name, bases, attrs):
        for method in attrs['methods']:
            attrs[method.__name__] = method # добавляем пары ключ - название метода, значение -  метод
        attrs.pop('methods') # удаляем methods из словаря атрибутов 
        return super().__new__(cls, name, bases, attrs)

`type` на самом деле тоже является метаклассом, который Python внутренне использует для создания всех классов

In [ ]:
new_classes = []
for i in range(1,5):
    new_classes.append(StudentMetaClass('Student%sYear'%str(i), (object, ), {'year': i, 'methods': student_methods})) 

In [ ]:
names = ['Vasya', 'Masha', 'Petya', 'Dasha']
for i, class_ in enumerate(new_classes):
    obj = class_(name=names[i])
    print(obj)
    print(obj.year)
    print(obj.introduce()+'\n')

### Аргумент metaclass

При написании класса можно добавить аргумент  metaclass, тогда питон при создании класса будет использовать указанный метакласс, а не type  
При указании metaclass питон 
+ перехватывает создание класса
+ изменяет класс
+ возвращает модифицированный объект класса

In [ ]:
class Student1Year(metaclass=StudentMetaClass):
    # задаем атрибуты, такие же как в словаре переданном StudentMetaClass последним аргументом
    year = 1 
    methods = student_methods

In [ ]:
student = Student1Year('Boris')

In [ ]:
student.introduce()

In [ ]:
student.year

In [ ]:
type(Student1Year)

## \_\_call\_\_

+ ```__call__()``` - определяет поведение, когда экземпляр класса вызывают (как функцию)

In [ ]:
class MyClass:
    pass
    
    # def __call__(cls):
    #     print('called method call')

my_class_instance = MyClass()

In [ ]:
my_class_instance()

![](https://i.stack.imgur.com/YVB4Q.png)

In [ ]:
class MyMeta(type):
    def __new__(cls, *args, **kwargs):
        print('called new of metaclass')
        return super().__new__(cls, *args, **kwargs)

    def __call__(cls, *args, **kwargs):
        print('called call of metaclass')
        return super().__call__(*args, **kwargs)

class MyClass(metaclass=MyMeta):
    def __init__(self):
        self.my_attr = 1
        print('init called')

    def __new__(self, *args, **kwargs):
        print('new called')
        return super().__new__(self, *args, **kwargs)

# MyClass() - тот самый __call__, мы вызываем экземпляр класса MyMeta (объект MyClass) как функцию
my_class = MyClass() 

**Задание**: 
   + написать метакласс, который переводит названия всех атрибутов и методов (кроме служебных) в верхний регистр
   + служебный = начинается и заканчивается на два нижих подчеркивания

In [ ]:
# пример работы
class MyClass(metaclass=UpperCaseMetaclass):
    attr1 = 1

In [ ]:
my_object = MyClass()
my_object.ATTR1
# 1

## Зачем нужны метаклассы - примеры

### Синглтон

Паттерн синглтон:
+ создание одного и только одного экземпляра класса
+ предоставление глобальной точки доступа к нему

Пример в питоне - None:

In [ ]:
a = None

In [ ]:
b = None

In [ ]:
a is b

In [ ]:
print(id(a), id(b), id(None))

In [ ]:
# можно делать так и наследоваться от него
class Singleton:
    def __new__(cls):
        if not hasattr(cls, 'instance'): # проверяем что существует только один экземпляр
            cls.instance = super(Singleton, cls).__new__(cls) # сохраняем в атрибуте класса информацию о созданном экземпляре
        return cls.instance
s = Singleton()
print("Object created", s)
s1 = Singleton() # точка доступа через создание экземпляра
print("Object created", s1)

In [ ]:
s is s1

Метакласс для создания синглтона:

In [ ]:
# можно создать меткласс
class MetaSingleton(type):
    _instances = {} # храним set созданных экземпляров всех классов 
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(MetaSingleton, cls).__call__(*args, **kwargs)
        return cls._instances[cls]

class ParentLogger:
    is_parent = True

# наследуемся как обычно, никаких изменений в механизме наследования не просиходит
class Logger(ParentLogger, metaclass=MetaSingleton):
    pass

class Something(metaclass=MetaSingleton):
    some_attr = 0

logger1 = Logger()
print(MetaSingleton._instances)
logger2 = Logger()
print(MetaSingleton._instances)
something1 = Something()
print(MetaSingleton._instances)
something2 = Something()
print(MetaSingleton._instances)

In [ ]:
print(logger1 is logger2)
print(something1 is something2)
print(logger1 is something2) # на всякий случай стоит упомянуть

**Задание:**
+ выяснить и рассказать мне, наследуется ли принадлежность к метаклассу
+ то есть будут ли дочерние классы например Logger иметь тот же метакласс

### ORM

**Object Relational Mapping** - отображение отношений реляционной бд в классы и объекты
![](https://www.fullstackpython.com/img/visuals/orms-bridge.png)

Примеры:
+ SQLAlchemy
+ DjangoORM
+ SQLObject
+ ClickhouseORM
+ PonyORM
+ и т.д. 

Зачем нужно:
+ упрощает написание запросов и работу с их результатами
    ![](https://www.dropbox.com/s/em0e38etaqmopgr/Screenshot%20from%202021-01-20%2001-38-55.png?dl=1)
+ более абстрактный и универсальный интерфейс
+ проще дебажить и тестировать
+ многие вещи автоматизированы


## Классы данных - dataclasses

+ python 3.7+
+ автоматизируют генерацию кода классов, использующихся для хранения данных
+ без использования метклассов
+ лучше использовать классы данных чем кортеж, словарь и т.д, удобнее получать доступ к атрибутам, код красивее, понятнее и проще для тестирования и отладки
+ [документация](https://docs.python.org/3/library/dataclasses.html)

In [ ]:
from dataclasses import dataclass

In [ ]:
# создаем класс игральной карты

@dataclass
class Card:
    rank: str
    suit: str

Допишет за нас по умолчанию:
+ ***init*** с заданными нами аргументами
+ красивый ***repr***
+ ***eq*** (элементы равны, если равны значения всех атрибутов)

In [ ]:
type(Card)

In [ ]:
# init
queen_of_hearts = Card(rank='Q', suit='heart')

In [ ]:
# красивый repr
queen_of_hearts

In [ ]:
another_queen_of_hearts = Card(rank='Q', suit='heart')

In [ ]:
# равенство
another_queen_of_hearts == queen_of_hearts

Без использования dataclass все то же самое выглядело бы вот так:

In [ ]:
class RegularCard
    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit

    def __repr__(self):
        return (f'{self.__class__.__name__}'
                f'(rank={self.rank!r}, suit={self.suit!r})')

    def __eq__(self, other):
        if other.__class__ is not self.__class__:
            return NotImplemented
        return (self.rank, self.suit) == (other.rank, other.suit)

Параметры с их значениями по умолчанию (подробнее можно почитать в документации):

In [ ]:
@dataclass(init=True, # добавить init  
           repr=True, # добавить repr
           eq=True, # добавить eq
           order=False, # добавить методы для сравнения и сортировки (__le__, __ge__ и т.д.)
           unsafe_hash=False, # настройик добавления метода __hash__
           frozen=False) # сделать класс неизменяемым
class Card:

    rank: str
    suit: str

Order - по умолчанию объекты сравниваются как кортежи из всех их атрибутов:

In [ ]:
@dataclass(order=True, frozen=True)
class Card:
    rank: str
    suit: str

In [ ]:
queen_hearts = Card('Q', 'hearts')
queen_spades = Card('Q', 'spades')
ten_hearts = Card('10', 'hearts')
ten_spades = Card('10', 'spades')
king_hearts = Card('K', 'hearts')
king_spades = Card('K', 'spades')

Вот так работает в варианте по умолчанию, но для полноценного представления колоды карт придется переписать:

In [ ]:
queen_hearts > ten_spades

In [ ]:
queen_spades > ten_hearts

In [ ]:
queen_spades < king_spades # неправильно

In [ ]:
for card in sorted([queen_hearts, queen_spades, ten_hearts, ten_spades, 
                    king_hearts, king_spades]):
    print(card)

Также легко добавить 
+ значение атрибута по умолчанию
+ атрибуты класса

In [ ]:
from typing import ClassVar

@dataclass
class Animal:
    fav_food: ClassVar = 'pizza' # переменная класса
        
    name: str
    # значения по умолчанию
    species: str = 'unknown'
    scariness: int = 0 
    legs: int = 4
    

In [ ]:
cat = Animal('Kitty','Cat')

In [ ]:
cat.legs

In [ ]:
cat.fav_food

In [ ]:
Animal.fav_food

Наследование происходит как обычно:

In [ ]:
@dataclass
class Cat(Animal):
    fav_food: ClassVar = 'mouse'
        
    species: str = 'Cat'
    whiskers: int = 28
    
    # можно определять свои методы
    def sound(self):
        if self.scariness > 10:
            return 'Grrr!'
        return 'Meow!'

In [ ]:
cat = Cat(name='Kitty')

In [ ]:
cat.species

In [ ]:
cat.fav_food

In [ ]:
cat.legs

In [ ]:
cat.whiskers

In [ ]:
cat.sound()

**Задание:**
   + сделать рабочий датакласс карт, чтобы можно было играть в пьяницу например
   + нормальное сравнение карт
   + карты разной масти, но одинакового достоинства равны
   + порядок возможных номиналов по возрастанию: ['6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
   + класс должен быть неизменяемым

In [ ]:
@dataclass
class Card:
    pass

Код для проверки:

In [ ]:
ranks = ['6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
suits = ['hearts', 'diamonds', 'spades', 'clubs']
deck = []

for suit in suits:
    for rank in ranks:
        deck.append(Card(rank=rank,suit=suit))

In [ ]:
deck

In [ ]:
sorted(deck)